In [1]:
import requests
from selenium import webdriver
from PIL import Image
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ChromeOptions
from selenium.webdriver import ActionChains
import time
import random

In [2]:
chrome_options=Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
option = ChromeOptions()
option.add_experimental_option('excludeSwitches',['enable-automation'])

In [3]:
url = "https://www.douban.com/"

In [4]:
driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe",)# chrome_options=chrome_options, options=option)
driver.get(url)

In [5]:
def input_username_password(username, password):
    #切换到iframe[0]标签
    driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[0])
    #切换到账号密码登录
    driver.find_element_by_class_name("account-tab-account").click()
    #输入账号
    driver.find_element_by_id("username").click()
    driver.find_element_by_id("username").send_keys(username)
    #输入密码
    driver.find_element_by_id("password").click()
    driver.find_element_by_id("password").send_keys(password)
    #点击登录
    driver.find_element_by_class_name("account-form-field-submit").click()

In [6]:
def get_img():
    #切换到iframe('tcaptcha_iframe')
    driver.switch_to.frame('tcaptcha_iframe')
    #获取缺货、完整图片的url
    src = driver.find_element_by_xpath('//*[@class="tc-bg"]/img').get_attribute('src')
    src_background = src.replace('img_index=1', 'img_index=0')
    #下载缺失的图片和完整的图片
    open('./img_canque.png', 'wb').write(requests.get(url=src).content)
    open('./img_wanzheng.png', 'wb').write(requests.get(url=src_background).content)
    print("下载完成！")
    #加载缺失、完整的图片
    captcha_queshi = Image.open('./img_canque.png')
    captcha_wanzheng = Image.open('./img_wanzheng.png')
    return captcha_queshi, captcha_wanzheng

In [19]:
def resize_img(img):
    #原图片为680×390px 渲染的大小为283×162px
    a = 2.403
    (x, y) = img.size
    x_resize = int(x // a)
    y_resize = int(y // a)
    img = img.resize((x_resize, y_resize), Image.ANTIALIAS)
    return img

In [8]:
def is_pixel_equal(img1, img2, x, y):
    pixel1, pixel2 = img1.load()[x, y], img2.load()[x, y]
    sub_index = 100
    if abs(pixel1[0] - pixel2[0]) < sub_index and abs(pixel1[1] - pixel2[1]) < sub_index and abs(pixel1[2] - pixel2[2]) < sub_index:
        return True
    else:
        return False

In [9]:
def get_gap_offest(img1, img2):
    distance = 70
    for i in range(distance, img1.size[0]):
        for j in range(img1.size[1]):
            if not is_pixel_equal(img1, img2, i, j):
                distance = i
                return distance
    return distance

In [10]:
def get_track(distance):
    distance -= 30
    a = distance / 4
    track = [a, a, a, a]
    return track

In [11]:
def shake_mouse():
    ActionChains(driver).move_by_offset(xoffset=-2, yoffset=0).perform()
    ActionChains(driver).move_by_offset(xoffset=2, yoffset=0).perform()

In [12]:
def operate_slider(track):
    slider_bt = driver.find_element_by_xpath('//*[@class="tc-drag-thumb"]')
    ActionChains(driver).click_and_hold(slider_bt).perform()
    for i in track:
        ActionChains(driver).move_by_offset(xoffset=i, yoffset=0).perform()
        time.sleep(random.random() / 100)
    time.sleep(random.random())
    back_tracks = [-1, -0.5, -1]
    for i in back_tracks:
        time.sleep(random.random() / 100)
        ActionChains(driver).move_by_offset(xoffset=i, yoffset=0).perform()
    shake_mouse()
    time.sleep(random.random())
    ActionChains(driver).release().perform()
    

In [26]:
def login():
    input_username_password('18939200916', 'z20011225')
    img1, img2 = get_img()
    img1 = resize_img(img1)
    img2 = resize_img(img2)
    distance = get_gap_offest(img1, img2)
    track = get_track(distance)
    operate_slider(track)